In [2]:
import graphlab
import math
import numpy as np
import random as rn
from matplotlib import pyplot as plt

In [3]:
rn.seed(91803)
n=30
x=graphlab.SArray([rn.random() for i in range(n)]).sort()

[INFO] This non-commercial license of GraphLab Create is assigned to ssubramanian90@gmail.com and will expire on February 08, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-10200 - Server binary: C:\Users\ssubr\Anaconda2\envs\dato-env\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\ssubr\AppData\Local\Temp\graphlab_server_1458921435.log.0
[INFO] GraphLab Server Version: 1.8.1


In [4]:
x

dtype: float
Rows: 30
[0.009095552722691913, 0.07966496214576213, 0.11784178322693395, 0.13769715723832776, 0.17727278878141373, 0.1916994311191108, 0.2026423442654708, 0.21786195415174336, 0.2487038089427649, 0.2512838032174758, 0.289243264751499, 0.2912068165523206, 0.3112290873841299, 0.3133675913563383, 0.3397060992788279, 0.36128016886670167, 0.4070925866477364, 0.4646585555449344, 0.5400057387608802, 0.5657030541964114, 0.7213150190287124, 0.7425473353140943, 0.7523266492398751, 0.7881591434738809, 0.8066314871194551, 0.8380061087249823, 0.842404220933413, 0.9127526366082569, 0.944606759803847, 0.955411707200171]

Develop y

In [5]:
y=x.apply(lambda x:math.sin(4*x))

Add noise to y

In [6]:
rn.seed(1)
e=graphlab.SArray([rn.gauss(0,1.0/3.0) for i in range(n)])
y=y+e

In [7]:
y

dtype: float
Rows: 30
[0.4657691028289499, 0.7964426955579553, 0.4762166904243817, 0.2685115081863658, 0.28708653023747643, 0.7042775644547797, 0.38397858486715386, 0.2863185778328726, 0.9050956726913648, 0.888692646577532, 1.097746233496042, 0.6140631563751847, 0.9490378029452767, 0.9284925996803823, 0.4756749447675509, 1.1714455970021094, 1.1052467742248486, 1.7552305749881358, 0.8990270968982104, 0.7217271571111419, 0.6644537289429158, 0.23682901468574138, 0.4349109106004362, -0.13289178455013648, -0.012107280541919035, 0.13254738362510274, 0.006029001718016486, -0.4448450505009305, -0.9554226689949985, -0.4804278908080471]

In [8]:
e

dtype: float
Rows: 30
[0.4293949177184876, 0.48314853623325693, 0.022111936312753967, -0.25484788365721056, -0.3640577383680471, 0.010444838943905622, -0.34070105667029094, -0.47894314836750995, 0.06643732549458461, 0.04445820155286828, 0.18215610011274386, -0.3046569812451042, 0.001668427875524148, -0.021580586790895962, -0.5019430004202472, 0.17933239288701125, 0.10690370033294941, 0.7963706810802287, 0.06765639243665474, -0.04823410272164017, 0.41091905834297454, 0.06626374939780849, 0.3030103420510697, -0.12184808874417226, 0.07272393778686082, 0.34142955949567266, 0.23208234081765383, 0.042824082877618797, -0.36076934153201484, 0.14840725745368283]

Defining plotting

In [9]:
data=graphlab.SFrame({'X1':x,'Y':y})

def plot_data(data):
    plt.plot(data['X1'],data['Y'],'k.')
    plt.xlabel('x')
    plt.ylabel('y')
    

In [10]:
data

X1,Y
0.00909555272269,0.465769102829
0.0796649621458,0.796442695558
0.117841783227,0.476216690424
0.137697157238,0.268511508186
0.177272788781,0.287086530237
0.191699431119,0.704277564455
0.202642344265,0.383978584867
0.217861954152,0.286318577833
0.248703808943,0.905095672691
0.251283803217,0.888692646578


In [11]:
plot_data(data)

#Define some useful polynomial regression functions

Define a function to create our features for a polynomial regression model of any degree

In [12]:
def polynomial_features(data, degree):
    datacopy=data.copy()
    for i in range(1, degree):
        datacopy['X'+str(i+1)]=datacopy['X'+str(i)]*datacopy['X1']
    return datacopy


Define a function to fit a polynomiallinear regression model of degree "deg" to the data in "data"

In [13]:
def polynomial_regression(data, degree):
    model=graphlab.linear_regression.create(polynomial_features(data, degree),target='Y',l2_penalty=0.,l1_penalty=0.,validation_set=None, verbose=False)
    return model

Define function to plot data and predictions made, since we are going to use it many times.

In [14]:
def plot_poly_predictions(data, model):
    plot_data(data)

    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1
    
    # Create 200 points in the x axis and compute the predicted value for each point
    x_pred = graphlab.SFrame({'X1':[i/200.0 for i in range(200)]})
    y_pred = model.predict(polynomial_features(x_pred,deg))
    
    # plot predictions
    plt.plot(x_pred['X1'], y_pred, 'g-', label='degree ' + str(deg) + ' fit')
    plt.legend(loc='upper left')
    plt.axis([0,1,-1.5,2])

Create a function that prints the polynomial coefficients in a pretty way :)

In [15]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    degree = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(degree) + ':'
    w.reverse()
    print np.poly1d(w)

#Fit polynomial regression models (degree-2)

we are fitting a 2nd degree polynomial to the data

In [16]:
model=polynomial_regression(data,2)

In [17]:
model.coefficients

name,index,value,stderr
(intercept),None,0.0382833304999,0.179256323069
X1,None,4.5149486792,0.887610375556
X2,None,-5.50721931537,0.851436821376


In [18]:
print_coefficients(model)

Learned polynomial for degree 2:
        2
-5.507 x + 4.515 x + 0.03828


In [19]:
plot_poly_predictions(data,model)

#Fit a degree-4 polynomial

In [20]:
model4=polynomial_regression(data,4)
print_coefficients(model4)
plot_poly_predictions(data,model4)

Learned polynomial for degree 4:
       4         3        2
26.22 x - 56.08 x + 33.4 x - 4.926 x + 0.6229


#Fit a degree-16 polynomial

In [21]:
model16=polynomial_regression(data,16)
print_coefficients(model16)
plot_poly_predictions(data,model16)

Learned polynomial for degree 16:
           16             15            14             13
-5.75e+06 x  + 2.164e+07 x  - 1.85e+07 x  - 2.934e+07 x 
              12             11             10             9
 + 6.199e+07 x  - 1.568e+07 x  - 5.103e+07 x  + 5.415e+07 x
              8             7             6             5            4
 - 1.327e+07 x - 1.217e+07 x + 1.171e+07 x - 4.655e+06 x + 1.03e+06 x
              3        2
 - 1.298e+05 x + 8653 x - 249.8 x + 2.113





#Ridge Regression





Ridge regression aims to avoid overfitting by adding a cost to the RSS term of standard least squares that depends on the 2-norm of the coefficients ∥w∥. The result is penalizing fits with large coefficients. The strength of this penalty, and thus the fit vs. model complexity balance, is controled by a parameter lambda (here called "L2_penalty").





Define our function to solve the ridge objective for a polynomial regression model of any degree:


In [22]:
def polynomial_ridge_regression(data, deg, l2_penalty):
    model = graphlab.linear_regression.create(polynomial_features(data,deg), 
                                              target='Y', l2_penalty=l2_penalty,
                                              validation_set=None,verbose=False)
    return model

#Perform a ridge fit of a degree-16 polynomial using a very small penalty strength


In [23]:
model16_2 = polynomial_ridge_regression(data, deg=16, l2_penalty=1e-25)
print_coefficients(model16_2)

Learned polynomial for degree 16:
           16             15            14             13
-5.75e+06 x  + 2.164e+07 x  - 1.85e+07 x  - 2.934e+07 x 
              12             11             10             9
 + 6.199e+07 x  - 1.568e+07 x  - 5.103e+07 x  + 5.415e+07 x
              8             7             6             5            4
 - 1.327e+07 x - 1.217e+07 x + 1.171e+07 x - 4.655e+06 x + 1.03e+06 x
              3        2
 - 1.298e+05 x + 8653 x - 249.8 x + 2.113


#Perform a ridge fit of a degree-16 polynomial using a very large penalty strength

In [24]:
model16_3 = polynomial_ridge_regression(data, deg=16, l2_penalty=100)
print_coefficients(model16_3)

Learned polynomial for degree 16:
         16          15          14         13          12          11
-0.1658 x  - 0.1621 x  - 0.1585 x  - 0.155 x  - 0.1516 x  - 0.1479 x 
          10          9          8          7          6          5
 - 0.144 x  - 0.1396 x - 0.1343 x - 0.1278 x - 0.1194 x - 0.1084 x
            4           3           2
 - 0.09384 x - 0.07429 x - 0.04825 x - 0.01687 x + 0.6768


In [25]:
plot_poly_predictions(data,model16_3)

#Let's look at fits for a sequence of increasing lambda values

In [26]:
for l2_penalty in [1e-25, 1e-10, 1e-6, 1e-3, 1e2]:
    model = polynomial_ridge_regression(data, deg=16, l2_penalty=l2_penalty)
    print 'lambda = %.2e' % l2_penalty
    print_coefficients(model)
    print '\n'
    plt.figure()
    plot_poly_predictions(data,model)
    plt.title('Ridge, lambda = %.2e' % l2_penalty)

lambda = 1.00e-25
Learned polynomial for degree 16:
           16             15            14             13
-5.75e+06 x  + 2.164e+07 x  - 1.85e+07 x  - 2.934e+07 x 
              12             11             10             9
 + 6.199e+07 x  - 1.568e+07 x  - 5.103e+07 x  + 5.415e+07 x
              8             7             6             5            4
 - 1.327e+07 x - 1.217e+07 x + 1.171e+07 x - 4.655e+06 x + 1.03e+06 x
              3        2
 - 1.298e+05 x + 8653 x - 249.8 x + 2.113


lambda = 1.00e-10
Learned polynomial for degree 16:
           16             15        14             13             12
4.636e+04 x  - 9.651e+04 x  + 6972 x  + 6.553e+04 x  + 2.517e+04 x 
              11             10             9             8        7
 - 3.809e+04 x  - 4.182e+04 x  + 1.193e+04 x + 4.017e+04 x - 5416 x
             6             5             4        3         2
 - 3.46e+04 x + 3.005e+04 x - 1.227e+04 x + 2875 x - 364.8 x + 19.54 x + 0.3422


lambda = 1.00e-06
Learned polyno




#Perform a ridge fit of a degree-16 polynomial using a "good" penalty strength





We will learn about cross validation later in this course as a way to select a good value of the tuning parameter (penalty strength) lambda. Here, we consider "leave one out" (LOO) cross validation, which one can show approximates average mean square error (MSE). As a result, choosing lambda to minimize the LOO error is equivalent to choosing lambda to minimize an approximation to average MSE.


In [27]:
#LOO- Returns the average MSE

def loo(data, degree, l2_penalty_values):
    #Create polynomial features
    polynomial_features(data, degree)
    
    # Create folds for cross validatation- number of folds= number of data points
    num_folds=len(data)
    folds=graphlab.cross_validation.KFold(data, num_folds)
    
    # for each value of l2_penalty, fit a model for each fold and compute average MSE
    l2_penalty_mse=[]
    min_mse=None
    best_l2_penalty=None
    for l2_penalty_value in l2_penalty_values:
        next_mse=0.0
        for train_set, validation_set in folds:
            #train model
            model=graphlab.linear_regression.create(train_set, target='Y', l2_penalty=l2_penalty_value, validation_set=None, verbose=False)
            
            #predict on validation set
            y_predicted=model.predict(validation_set)

            #compute squared error
            next_mse +=((y_predicted-validation_set['Y'])**2).sum()
        
        # save squared error in list of MSE for each l2_penalty
        next_mse = next_mse/num_folds
        l2_penalty_mse.append(next_mse)
        if min_mse is None or next_mse < min_mse:
            min_mse = next_mse
            best_l2_penalty = l2_penalty_value
    
    return l2_penalty_mse,best_l2_penalty    

Run LOO cross validation for "num" values of lambda, on a log scale

In [28]:
l2_penalty_values = np.logspace(-4, 10, num=10)

In [29]:
l2_penalty_values

array([  1.00000000e-04,   3.59381366e-03,   1.29154967e-01,
         4.64158883e+00,   1.66810054e+02,   5.99484250e+03,
         2.15443469e+05,   7.74263683e+06,   2.78255940e+08,
         1.00000000e+10])

In [30]:
l2_penalty_mse,best_l2_penalty = loo(data, 16, l2_penalty_values)

In [31]:
l2_penalty_mse

[0.23138098383654065,
 0.23137349679013852,
 0.23113535636203864,
 0.23974667561854116,
 0.31456516408512686,
 0.3233404978495715,
 0.32360301408259506,
 0.323610333882086,
 0.3236105375715853,
 0.32361054323937594]

In [32]:
best_l2_penalty

0.12915496650148839

Plot results of estimating LOO for each value of lambda

In [33]:
plt.plot(l2_penalty_values,l2_penalty_mse,'k-')
plt.xlabel('$\L2_penalty$')
plt.ylabel('LOO cross validation error')
plt.xscale('log')
plt.yscale('log')

In [34]:
model16_4 = polynomial_ridge_regression(data, deg=16, l2_penalty=best_l2_penalty)
print_coefficients(model16_4)

Learned polynomial for degree 16:
         16          15          14          13         12          11
0.01707 x  + 0.2724 x  + 0.4717 x  + 0.6001 x  + 0.642 x  + 0.5826 x 
           10          9         8          7         6         5
 + 0.4099 x  + 0.1187 x - 0.282 x - 0.7571 x - 1.227 x - 1.543 x
          4          3          2
 - 1.473 x - 0.7397 x + 0.6641 x + 1.425 x + 0.3663





#Lasso Regression





Lasso regression jointly shrinks coefficients to avoid overfitting, and implicitly performs feature selection by setting some coefficients exactly to 0 for sufficiently large penalty strength lambda (here called "L1_penalty"). In particular, lasso takes the RSS term of standard least squares and adds a 1-norm cost of the coefficients ∥w∥.





Define our function to solve the lasso objective for a polynomial regression model of any degree:


In [35]:
def polynomial_lasso_regression(data, deg, l1_penalty):
    model = graphlab.linear_regression.create(polynomial_features(data,deg), 
                                              target='Y', l2_penalty=0.,
                                              l1_penalty=l1_penalty,
                                              validation_set=None, 
                                              solver='fista', verbose=False,
                                              max_iterations=3000, convergence_threshold=1e-10)
    return model




#Explore the lasso solution as a function of a few different penalty strengths





We refer to lambda in the lasso case below as "l1_penalty"


In [36]:
for l1_penalty in [0.0001, 0.01, 0.1, 10]:
    model = polynomial_lasso_regression(data, deg=16, l1_penalty=l1_penalty)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (model.coefficients['value']).nnz()
    print_coefficients(model)
    print '\n'
    plt.figure()
    plot_poly_predictions(data,model)
    plt.title('LASSO, lambda = %.2e, # nonzeros = %d' % (l1_penalty, (model.coefficients['value']).nnz()))

l1_penalty = 1.000000e-04
number of nonzeros = 17
Learned polynomial for degree 16:
       16         15         14         13         12         11
42.82 x  + 3.809 x  - 18.37 x  - 27.14 x  - 24.48 x  - 13.87 x 
           10         9         8         7         6         5
 + 0.1423 x  + 14.27 x + 22.78 x + 21.43 x + 8.656 x - 11.34 x
          4         3        2
 - 25.38 x - 12.61 x + 23.2 x - 4.328 x + 0.6297


l1_penalty = 1.000000e-02
number of nonzeros = 16
Learned polynomial for degree 16:
         16         15          14            13          11
-0.7469 x  - 2.804 x  - 0.8283 x  - 0.002653 x  + 0.2012 x 
             10       9         8          7          6          5
 - 0.002729 x  + 1.7 x + 8.727 x + 0.8003 x + 0.2017 x - 0.1724 x
          4           3         2
 - 16.11 x - 0.01174 x + 7.863 x - 0.6322 x + 0.4429


l1_penalty = 1.000000e-01
number of nonzeros = 5
Learned polynomial for degree 16:
       15           14         4
1.448 x  + 0.02838 x  - 4.266 x + 1

Above: We see that as lambda increases, we get sparser and sparser solutions. However, even for our non-sparse case for lambda=0.0001, the fit of our high-order polynomial is not too wild. This is because, like in ridge, coefficients included in the lasso solution are shrunk relative to those of the least squares (unregularized) solution. This leads to better behavior even without sparsity. Of course, as lambda goes to 0, the amount of this shrinkage decreases and the lasso solution approaches the (wild) least squares solution.